In [4]:
import pandas as pd
from lib import standardize_item_numbers

In [5]:
def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["rtcc_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [6]:
df = clean()

C:\Users\PC\AppData\Local\Temp\ipykernel_20224\153542535.py:2: DtypeWarning: Columns (14,20,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [7]:
df = df.drop_duplicates(subset=["offender_race", "offender_gender", "offender_age", "item_number"])

In [8]:
df['occurred_date_time'] = sorted(pd.to_datetime(df['occurred_date_time']))
df['hour'] = df['occurred_date_time'].dt.hour
df = df[~((df.hour.fillna("") == ""))]
df.loc[:, "hour"] = df.hour.astype(int)

In [9]:
df.hour.value_counts()

1     528
23    487
2     474
0     461
21    454
20    448
19    431
3     425
17    423
18    412
22    403
16    381
15    370
11    336
14    330
4     328
12    303
10    286
13    259
9     208
5     202
8     192
6     185
7     162
Name: hour, dtype: int64

In [10]:
df.to_csv("../data/real_time_crime_center/requests_by_hour_of_the_day.csv")